In [ ]:
from util import *

flag='small'
playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,playlist_tracks_tru,tracks,playlists,playlist_tracks = load_small_data()
y_true=ytrue(playlists_tst,playlist_tracks_tru)

track_vecs=np.load('raw/w2v/track_als_vecs.npy')

In [ ]:
track_vecs_tra=track_vecs[tracks.track_id]
tid_tra=tracks.track_id.values
emb_dim=track_vecs.shape[1]

In [ ]:
from keras.layers.embeddings import Embedding

def build_embedding_layer():
    vecs=track_vecs
    len_dim=vecs.shape[0]+1
    emb_dim=vecs.shape[1]
    
    emb_matrix=np.zeros((len_dim,emb_dim))
    emb_matrix[:-1,:]=vecs
    
    emb_layer=Embedding(len_dim,emb_dim,trainable=False)
    emb_layer.build((None,))
    emb_layer.set_weights([emb_matrix])
    
    return emb_layer

embedding_layer = build_embedding_layer()
assert abs(embedding_layer.get_weights()[0][1][3] - track_vecs[1][3])<1e-3

In [ ]:
import numpy as np
np.random.seed=0
from keras.models import Model,Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, SimpleRNN, InputLayer, Lambda, Reshape
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed=1
max_len=100

In [ ]:
X=np.zeros((playlists_tra.shape[0],max_len),dtype='int')
Y=np.zeros((playlists_tra.shape[0],max_len,emb_dim))

for i in tnrange(playlists_tra.shape[0]):
    pid=playlists_tra.iloc[i]['pid']
    X[i][0]=pid
    tracks=playlist_tracks_tra[playlist_tracks_tra.pid==pid].track_id.values
    for j in range(max_len):
        if j<tracks.shape[0]-1:
            X[i][j]=tracks[j]
            Y[i][j]=tracks[j+1]
    
Y=Y.reshape(Y.shape[1],Y.shape[0],Y.shape[2])

In [ ]:
import numpy as np
def l2_norm_2(x, axis=None):
    """
    takes an input tensor and returns the l2 norm along specified axis
    """

    square_sum = np.sum(np.square(x), axis=axis, keepdims=True)
    norm = np.sqrt(np.maximum(square_sum, 1e-4))

    return norm

x=np.random.rand(emb_dim,1)
a=np.dot(track_vecs,x)/(l2_norm_2(track_vecs,axis=1)*l2_norm_2(x))
np.argmax(a)

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer

class PlaylistGenerator:
    def __init__(self,n_a,emb_dim,max_len):
        self.n_a = n_a
        X=Input(shape=(max_len,))
        a0=Input(shape=(n_a,))
        c0=Input(shape=(n_a,))
        outputs = []

        reshapor1=Reshape((1,emb_dim),name='reshape1')
        reshapor2=Reshape((emb_dim,1),name='reshape2')
        self.rnn_cell = SimpleRNN(n_a,return_state=True)
        self.densor=Dense(emb_dim)

        a = a0
        c = c0
        for i in range(max_len):
            x=Lambda(lambda x:x[:,i])(X)
            x=embedding_layer(x)
            
            x=reshapor1(x)
            
            a,_=self.rnn_cell(x,initial_state=[a])
            o=self.densor(a)
            outputs.append(o)

        self.model_train=Model(inputs=[X,a0],outputs=outputs)
#         self.model_train.summary()
        
        self.model_train.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
        
        x0 = Input(shape=(emb_dim,))
        a0 = Input(shape=(n_a,),name='a0')
        c0 = Input(shape=(n_a,),name='c0')
        a = a0
        c = c0
        x = x0

        x=reshapor1(x)
        a,_=self.rnn_cell(x,initial_state=[a])
        o=self.densor(a)
            
        self.model_infer=Model(inputs=[x0,a0],outputs=[o,a])
#         self.model_infer.summary()

        self.tv_l2=l2_norm_2(track_vecs_tra,axis=1)
        
    def fit(self,X,Y,epochs):
        a0=np.zeros((X.shape[0],self.n_a))
        self.model_train.fit([X,a0],list(Y),epochs=epochs,batch_size=64)
        
        
        
    def predict(self,X):
        a=np.zeros((1,self.n_a))
        
        results=[]
        for i in range(len(X)):
            tv,a=self.model_infer.predict([track_vecs[X[i]].reshape(1,emb_dim),a])
            
        if len(X)>1:
            tv=track_vecs[X[-1]]
            
        for i in range(500):
            tv,a=self.model_infer.predict([tv.reshape(1,emb_dim),a])
            idxs=np.argsort((np.dot(track_vecs_tra,tv.T)/(self.tv_l2*l2_norm_2(tv.T))).reshape((track_vecs_tra.shape[0],)))
#             print(idxs[-10:])
            for j in range(len(idxs)-1,-1,-1):
                tid=tid_tra[idxs[j]]
                if tid not in results and tid not in X:
                    results.append(tid)
                    tv=track_vecs[tid]
                    break
            
                    
        return results
    
n_a=256
model=PlaylistGenerator(n_a,emb_dim,max_len)
    

In [ ]:
model.fit(X,Y,epochs=5)

In [ ]:
y_pred={}
ypids=[]
for i in tnrange(playlists_tst.shape[0]):
    pid=playlists_tst.iloc[i]['pid']
    track_ids=playlist_tracks_tst[playlist_tracks_tst.pid==pid].track_id.values
    x=list(track_ids)
    if len(x)>0:
        y=model.predict(x)
        y_pred[pid]=y
    else:
        ypids.append(pid)

In [ ]:
from copy import deepcopy
y_true2=deepcopy(y_true)
for pid in ypids:
    del y_true2[pid]

In [ ]:
assert y_pred.keys()==y_true2.keys()
print('r_prec %.3f, ndcg %.3f, click %.3f'%(r_prec(y_true,y_pred),ndcg(y_true,y_pred),clicks(y_true,y_pred)))

In [ ]:
y_pred.keys()

In [ ]:
y_pred[90114][:10]

In [ ]:
y_pred[417812][:10]